# Moveworks Take Home Assignment 

In [1]:
#Importation of the necessary libraries
import pandas as pd
#import streamlit as st
from datetime import timedelta
import datetime
#import plotly_express as px

In [2]:
df_f1 = pd.read_csv('dashboard_tha_file1_shared.csv')

df_f2 = pd.read_csv('dashboard_tha_file2_shared.csv')

FileNotFoundError: [Errno 2] File b'dashboard_tha_file1_shared.csv' does not exist: b'dashboard_tha_file1_shared.csv'

## Metric 1: DAU_MAU ratio

#### Description:

#### Python Code:

In [14]:
def add_days_to_date(date, days):
    """Add days to a date and return the date.

    Args:
        date (string): Date string in YYYY-MM-DD format.
        days (int): Number of days to add to date

    Returns:
        date (date): Date in YYYY-MM-DD with X days added.
    """

    added_date = pd.to_datetime(date) + timedelta(days=days)
    added_date = added_date.strftime("%Y-%m-%d")

    return added_date

In [15]:
dic_date = {} 

for date_4 in df_f2.sort_values(by = ['date']).groupby("date").count().reset_index()["date"]:
    if (add_days_to_date(date_4, 30) in df_f2['date'].values): #Checking whether or not the date 30 days later than the date we are currently looking at is available in the dataset.
        d_dau = len(df_f2[df_f2["date"] == date_4].groupby("user_id").nunique())
        d_mau = len(df_f2[(df_f2["date"] >=  date_4) & (df_f2["date"] <= add_days_to_date(date_4, 30))].sort_values(by = 'date').groupby("user_id").nunique())
        dau_mau = (d_dau / d_mau) * 100 #Calculation of the DAU_MAU ratio
        dic_date[date_4] = round(dau_mau,1) #dictionary containing the dates and their respective dau_mau ratio

In [37]:
dat = list(dic_date.keys())
ratio = list(dic_date.values())

df = pd.DataFrame(dict(                          #Creation of a Dataframe in order to use streamlit to make a line chart.
    Day = dat,
    Ratio = ratio
))

#### Streamlit Code:

## Metric 2: Retention Rate

#### Description:

#### Python Code:

In [19]:
e_l_1 = []
e_l_2 = []
dic_seven = {}  #Dictionary containing the dates and their 7-day retention rate
dic_thirty = {} #Dictionary containing the dates and their 30-day retention rate

In [51]:
for date_3 in df_f2.sort_values(by = ['date']).groupby("date").count().reset_index()["date"]:
    if (add_days_to_date(date_3, 7) in df_f2['date'].values):
        e_l_1 = list(df_f2[df_f2["date"] == date_3]['user_id']) #list of all the user_ids of the date we are currently looking at.
        l_1 = list(df_f2[df_f2["date"] == add_days_to_date(date_3, 7)]['user_id']) #Same but for the date 30 days later.
        l_inter = list(set(e_l_1).intersection(l_1)) #Check the mutual user_ids 
        ret_rate = (len(l_inter) / len(e_l_1)) * 100 #Calculation of the retention rate.
        dic_seven[date_3] = str(round(ret_rate,2)) + "%"
        e_l_1 = []

In [21]:
def seven_day_ret(d): #Function outputing the 7-Day retention rate for a specific day
    return dic_seven[d]

In [22]:
for date_1 in df_f2.sort_values(by = ['date']).groupby("date").count().reset_index()["date"]: #go through all the dates
    if (add_days_to_date(date_1, 30) in df_f2['date'].values): #Checking whether or not the date 30 days later than the date we are currently looking at is available in the dataset.
        e_l_2 = list(df_f2[df_f2["date"] == date_1]['user_id']) #list of all the user_ids of the date we are currently looking at.
        l_2 = list(df_f2[df_f2["date"] == add_days_to_date(date_1, 30)]['user_id']) #same but for the date 30 days later.
        l_inter_1 = list(set(e_l_2).intersection(l_2)) #Find the common elements between the two lists
        ret_rate_1 = (len(l_inter_1) / len(e_l_2)) * 100 #retention rate = ratio between the common users between the two lists above and the original number of users in the day we are currently looking at
        dic_thirty[date_1] = str(round(ret_rate_1,2)) + "%" #add to the dictionary the calculated retention rate
        e_l_2 = []
        num_30 = 0

In [23]:
def thirty_day_ret(d): #Function outputing the 30-Day retention rate of the day we are interested in
    return dic_thirty[d] 

#### Streamlit Code:

## Metric 3: Daily Active Users (DAU)

#### Description:

#### Python Code:

In [24]:
def dau(date): #Function Calculating the Daily Active Users.
    dau = df_f2[df_f2['date'] == date]['user_id'].nunique()
    return dau 

In [25]:
dic_dau = {} #Dictionary containing a Date and its DAU.

for d in df_f2.sort_values(by = ['date']).groupby("date").count().reset_index()["date"]:
    dic_dau[d] = dau(d)


date_dau = list(dic_dau.keys())
nb_dau = list(dic_dau.values())

In [26]:
df_mau = pd.DataFrame(dict( #Creation of a Dataframe for using Streamlit.
    Day = date_dau,
    DAU = nb_dau
))

#### Streamlit Code:

## Metric 4: Weekly Active Users (WAU)

#### Description:

#### Python Code:

In [ ]:
df_f2['week'] = pd.DatetimeIndex(df_f2['date']).week #Converting our original dates to weeks and creating a new column in df_f2 containg each date's corresponding week.
for w in range(len(df_f2['week'])):
    df_f2['week'][w] = df_f2['week'][w] - 22 #Substracting 22 to show the corresponding week since the creation 
                                                #of the app and not the corresponding week of the date. Week 1 is 
                                                #the first week of the app's creation.

df_f2.sort_values(by = 'date')
m_w = df_f2.groupby('week').count().reset_index().rename(columns={'week': 'week', 'date': '# of users'})  #Count the number of weekly users.
m_w = m_w[['week','# of users']]


#### Streamlit Code:

## Metric 5: Monthly Active Users (MAU)

#### Description:

#### Python Code:

In [ ]:
df_f2['month'] = pd.DatetimeIndex(df_f2['date']).month #Converting our original dates to months and creating a new column in df_f2 containg each date's corresponding month.
df_month = df_f2.groupby('month').count().reset_index().rename(columns={'is_new_user': '# of visits', 'date': '# of users'})
df_month = df_month[['month','# of users']]
df_month["month"] = ['June','July','August','September'] #df_month is a dataframe with the months available in our Dataframes and their respective # of users.

#### Streamlit Code:

## Metric 6: Users' usage frequency for 30 days

#### Description:

#### Python Code:

In [31]:
df_new_user = df_f2[df_f2['is_new_user'] == True]

In [ ]:
def freq(date_chosen): #Function calculating the  the number of times users of a specific cohort used the app for a duration of 30 days. 
    dic_freq = {} 
    list_perc = []
    if (add_days_to_date(date_chosen, 30) in df_f2['date'].values): #Checking whether or not the date 30 days later than the date we are currently looking at is available in the dataset.
        list_uid = list(df_new_user[df_new_user["date"] == date_chosen]['user_id'].unique())
        for l in list_uid:
            dic_freq[l] = 0 

        df_period = df_f2[(df_f2["date"] >= date_chosen) & (df_f2["date"] <= add_days_to_date(date_chosen, 30))]['user_id'] #new dataframe which contains a period of thirty days.
        for u in df_period:
            if u in dic_freq: #Check how many times the users in the cohort are present in the dataframe which corresponds to its usage frequency.
                dic_freq[u] += 1

        data_items = list(dic_freq.keys()) #Converting the dictionary's keys to a column in the new dataframe below.
        data_value = list(dic_freq.values()) #Converting the dictionary's values to another column in the new dataframe below.

        df_freq = pd.DataFrame({
            '#_users': data_items,
            'frequency': data_value,
        })

        df_freq = df_freq.groupby("frequency").count().reset_index() #this steps calculates the total frequency for the cohort we are looking at.
        for x in list(df_freq["#_users"]):
            list_perc.append(round((x / sum(df_freq["#_users"])) * 100, 3)) #Transform the frequency into a percentage.

        df_freq["% of users"] = list_perc
        return df_freq #returns a dataframe with different frequencies, the number of users for those frequencies as well as the percentage each frequency represents.



#### Streamlit Code:

## Metric 6*: Average number of visits for 30 days.

#### Description:

#### Python Code:

In [ ]:
i = 0

for x in range(0, len(df_freq)):
    i += df_freq["frequency"][x] * df_freq["#_users"][x] 

avg_nb_visits = round(i / sum(df_freq["#_users"]), 2) #calculates the average number of times users in a specific cohort visited the app.

#### Streamlit Code:

## Metric 7: Features usage

#### Description:

#### Python Code:

In [45]:
m_f = df_f1.groupby("feature_used").count().reset_index().rename(columns={'timestamp': '# of uses', 'user_id': '# of users'})
m_f = m_f[['feature_used','# of users']]
m_f = m_f.sort_values(by = '# of users', ascending = False)
m_f_test = m_f.groupby('feature_used')['# of users'].sum() #Sum the total number of users who used the features.

#### Streamlit Code:

## Metric 8: Total number of users using the app per hour of the day

#### Description:

#### Python Code:

In [35]:
df_f1['hour'] = pd.DatetimeIndex(df_f1['timestamp']).hour #Taking only the hour during which the users checked the app.
m_h = df_f1.groupby('hour').count().reset_index().rename(columns={'hour': 'hour', 'timestamp': '# of users'})
m_h = m_h[['hour','# of users']]
m_h = m_h.sort_values(by = '# of users', ascending = False)
m_h_test = m_h.groupby('hour')['# of users'].sum().sort_values() #Sum the number of users who used the app for each hour over the entire duration.

#### Streamlit Code:

## Metric 9: Total number of users using the app per day of the week

#### Description:

#### Python Code:

In [36]:
df_f2['day'] = pd.to_datetime(df_f2['date']).dt.day_name() #Taking only the name of the day (ex: Monday, tuesday, etc...) during which the users checked the app.
df_f2.head()
m_d = df_f2.groupby("day").count().reset_index().rename(columns={'date': '# of users'})
m_d = m_d[['day','# of users']]
m_d.sort_values(by = 'day')
m_d["day"] = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
m_d["# of users"] = [4076,4132,4001,3932,3948,4009,4636]
m_d_test = m_d.groupby('day')['# of users'].sum()  #Sum the number of users who used the app for each day of the week over the entire duration.

#### Streamlit Code: